# credit anxiety index

this index combines objective market data with psychological indicators to detect *early signs of economic slowdown*.

## what is credit anxiety index?

an index that measures *financial tension* in the market by combining credit spread and public interest.

- *credit spread* — difference between yields on risky and safe bonds
- *public interest* — how many people actively search for information about credit problems

## why is it important?
during economic crises, investors become cautious, demand higher risk premium, and the public actively searches for information about financial problems.

## what is risk premium?

*risk premium* = additional return that investors require.

### example

| bond type | yield | risk |
|-----------|-------|------|
| government bonds (gs10) | 4% | low - government backed |
| corporate bonds (dbaa) | 6% | high - depends on corporation |
| risk premium | 2% | difference = compensation for risk |

### how it works:

investor says: "ok, i'll buy this riskier corporate bond, but i require *2% more* as compensation for the risk that the corporation might not pay back."

### premium dynamics  

when economy worsens:
- higher corporate bankruptcy risk
- investors demand higher premium
- spread rises (e.g. from 2% to 4%)

when stable:
- lower risk
- lower premium
- spread falls (e.g. from 4% back to 2%)


### in context of credit anxiety index
```
credit spread = dbaa - gs10 = credit risk premium
```


In [40]:
import urllib.request
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from pytrends.request import TrendReq

warnings.filterwarnings("ignore", category=FutureWarning)

today = pd.Timestamp('today').normalize()


### downloading data

*fred (federal reserve economic data)* is a database of the us federal reserve containing thousands of economic time series.

### we download:
- *gs10* - 10-year us treasury bonds
- *dbaa* - moody's seasoned baa corporate bond yield

In [41]:
today_str = dt.date.today().strftime("%Y-%m-%d")

url_dbaa = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id=DBAA&cosd=2020-01-01&coed={today_str}"
url_gs10 = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id=GS10&cosd=2020-01-01&coed={today_str}"

_=urllib.request.urlretrieve(url_dbaa, "DBAA.csv")
_=urllib.request.urlretrieve(url_gs10, "GS10.csv")

### gs10 (10-year treasury)
- gold standard of "risk-free" investment
- backed by us government
- during crises, investors flee to safety, yield falls


### dbaa (baa corporate bonds)
- bonds from corporations with *lower credit rating* (baa/bbb)
- considered *riskier* than government bonds
- when economy weakens, investors demand higher premium for taking risk, yield rises

In [42]:
dbaa = (pd.read_csv("DBAA.csv",
                    parse_dates=['observation_date'],
                    date_format='%m/%d/%y')  
          .rename(columns={'DBAA':'rate'}))

gs10 = (pd.read_csv("GS10.csv",
                    parse_dates=['observation_date'],
                    date_format='%m/%d/%y')
          .rename(columns={'GS10':'rate'}))

# zadnji dostupni dan
r_dbaa = dbaa.dropna().iloc[-1]
r_gs10 = gs10.dropna().iloc[-1]

spread = float(r_dbaa.rate) - float(r_gs10.rate)
print(f"DBAA: {r_dbaa.rate} % | GS10: {r_gs10.rate} % | Spread: {spread:.2f} %")

DBAA: 5.85 % | GS10: 4.09 % | Spread: 1.76 %


### what is credit spread?

#### credit spread = dbaa - gs10

*spread shows credit risk premium:*

| spread | status | description |
|--------|--------|-------------|
| **~1-2%** | low | investors relaxed, willing to take risk, stable economy |
| **2-3%** | moderate | increased caution, first concerns about creditworthiness |
| **>4%** | high | panic, investors fleeing risky bonds, possible recession signs |

*historical context:*
- normal times (2015-2019): ~2%
- covid crisis (march 2020): ~4%
- financial crisis (2008-2009): ~6%

*why does spread widen during crises?*

when economy weakens:
1. corporations face higher bankruptcy probability → investors sell corporate bonds → dbaa rises
2. investors seek safety → buy treasury bonds → gs10 falls
3. *result*: spread explodes


In [43]:
#pip install pytrends 


### fetching google trends data

we track how many people search for *"credit crunch"* - an indicator of public concern.

*why measure public interest in "credit crunch"?*

**credit crunch** = situation when banks suddenly reduce credit availability

### market psychology:
- when economy turns down, people start googling "credit crunch", "recession", "layoffs"
- *rise in searches = rise in concern* = possible early problem signal
- sometimes public senses problems *before they appear in official data*

### google trends scoring:

| value | status |
|-------|--------|
| **<20** | low concern |
| **20-50** | moderate concern |
| **>50** | high concern, something is happening |

*why 3 months?*
`timeframe='today 3-m'` is short enough to catch trends, long enough to avoid noise.

In [44]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [45]:
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)
pytrends.build_payload(["credit crunch"], timeframe='today 3-m')
trend_df = pytrends.interest_over_time()
gt = trend_df["credit crunch"].iloc[-1]
print("Google Trends:", gt)

Google Trends: 41


### calculating anxiety index

we combine credit spread and public interest:

#### formula
```
anxiety index = credit spread + (google trends / 100)
```

- *credit spread*: objective measure of market risk
- *google trends*: subjective measure of public perception


*why this combination?*

| component | type | weight | description |
|-----------|------|--------|-------------|
| **credit spread** | objective | ~1.5-6 points | based on real market transactions |
| **google trends / 100** | subjective | ~0.2-1 point | captures psychological trends and perception |

*example:*
- spread = 2.5%
- google trends = 40
- *index = 2.5 + 0.4 = 2.9* → moderate tension

In [46]:
index = spread + gt/100
row   = pd.DataFrame([{"observation_date": today, "dbaa": r_dbaa.rate, "gs10": r_gs10.rate,
                       "spread": spread, "gtrend": gt, "anxiety_index": index}])


### saving and visualization

adding new data point and displaying trend.

In [47]:
try: hist = pd.read_csv("anxiety.csv", parse_dates=["observation_date"])
except: hist = pd.DataFrame()
hist  = pd.concat([hist, row]).drop_duplicates(subset=["observation_date"])
hist.to_csv("anxiety.csv", index=False)
print("Zadnji index:", round(index,2))

Zadnji index: 2.17


## interpretation of results

| index | status | economic situation |
|-------|--------|-------------------|
| **< 2** | calm market | stable economy, low risk premium, relaxed investors |
| **2-4** | moderate tension | increased caution, first concerns, expensive credit |
| **> 4** | high stress | serious crisis, panicked investors, possible recession signal |

### what does rising index mean?

*for corporations:*
- more expensive borrowing → less investment
- difficult access to capital → business slowdown
- lower profit → possible layoffs

*for investors:*
- "flight to quality" → fleeing to safe bonds
- higher market volatility
- panic selling of risky assets

*for economy:*
- early sign of slowdown before it appears in gdp
- reduced lending → less consumption
- self-fulfilling prophecy: fear → less investment → recession

### historical context

| event | spread | index | outcome |
|-------|--------|-------|---------|
| **2008 financial crisis** | ~6% | ~6.5 | severe recession |
| **2020 covid** | ~4% | ~4.3 | quick recovery with fed intervention |
| **2025** | ~2% | ~2 | relatively calm |

<br>

> **note**: thresholds are indicative. for more accurate boundaries, analysis of historical recessions (2008, 2020) and comparison with actual economic events is needed.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

max_idx = hist['anxiety_index'].max()

# Zone 
fig.add_hrect(y0=0, y1=2, fillcolor="green", opacity=0.15, 
              line_width=0, layer="below")
fig.add_hrect(y0=2, y1=4, fillcolor="orange", opacity=0.15, 
              line_width=0, layer="below")
fig.add_hrect(y0=4, y1=10, fillcolor="red", opacity=0.15,
              line_width=0, layer="below")

# hover info
fig.add_trace(go.Scatter(
    x=hist['observation_date'],
    y=hist['anxiety_index'],
    mode='lines+markers',
    name='Anxiety Index',
    line=dict(color='#1f77b4', width=2.5),
    marker=dict(size=6),
    hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Anxiety: %{y:.3f}<extra></extra>'
))

# dummy trace za legendu
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(size=12, color='green', opacity=0.15, symbol='square'),
    name='Mirno tržište (0-2)',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(size=12, color='orange', opacity=0.15, symbol='square'),
    name='Umjeren stres (2-4)',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(size=12, color='red', opacity=0.15, symbol='square'),
    name='Visok stres (4+)',
    showlegend=True
))

fig.add_hline(y=2, line_dash="dash", line_color="orange", 
              line_width=1.5, opacity=0.7)
fig.add_hline(y=4, line_dash="dash", line_color="red", 
              line_width=1.5, opacity=0.7)

last_date = hist['observation_date'].iloc[-1]
last_val = hist['anxiety_index'].iloc[-1]
fig.add_trace(go.Scatter(
    x=[last_date],
    y=[last_val],
    mode='markers+text',
    marker=dict(size=15, color='red', line=dict(width=2, color='black')),
    text=[f'{last_val:.2f}'],
    textposition="top center",
    textfont=dict(size=12, color='black'),
    name='Zadnja vrijednost',
    showlegend=False,
    hovertemplate='<b>Zadnja vrijednost</b><br>%{x|%Y-%m-%d}<br>Anxiety: %{y:.3f}<extra></extra>'
))

fig.update_layout(
    title=dict(
        text='Credit Anxiety Index',
        font=dict(size=20, color='black'),
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Datum',
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5
    ),
    yaxis=dict(
        title='Anxiety Index',
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        range=[0, 5]
    ),
    hovermode='closest',
    plot_bgcolor='white',
    height=600,
    width=1200,
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='gray',
        borderwidth=1
    )
)

# interaktivni HTML
fig.write_html("graf.html")
fig.show()